# Initial AI Agent PPO w/o
NN (working) MR2 Drag Disc

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

In [1]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 600
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = True
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 13,
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 0 is HS, 1 is 400m
  'carChoice' : 0, # 0 is MR2, 1 is Supra, 2 is Civic
  'rewardMode' : "simplex"
}

In [2]:
import ray
ray.shutdown()
ray.init()

2023-10-22 13:03:13,595	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.9
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
config = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        #batch_mode="completed_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=600,
        )
)

In [ ]:
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        try:
            json.dumps(obj)
        except TypeError:
            return "Not serializable object!"
        return obj

config_dict = config.to_dict()

print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

In [ ]:
algo = config.build()

In [ ]:
# algo.get_policy().model

In [ ]:
algo.train()

In [ ]:
save = algo.save()

In [ ]:
path_to_checkpoint = save.checkpoint.path

In [ ]:
N = 3000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()

#Works!

In [4]:
from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-21_21-02-14mxhz6gxj/checkpoint_000771")

2023-10-22 13:03:25,509	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-21_21-02-14mxhz6gxj/checkpoint_000771! Trying to extract checkpoint info from other files found in that dir.
2023-10-22 13:03:25,601	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=7532) GT Real Time instantiated
(RolloutWorker pid=7532) GT AI Server instantiated for rtgym
(RolloutWorker pid=7532) starting up on localhost port 9999
(RolloutWorker pid=7532) Waiting for a connection
(RolloutWorker pid=7532) Connection from ('127.0.0.1', 51057)
(RolloutWorker pid=7532) J:\git\TensorFlowPSX\Py\dragSpaced.csv
(RolloutWorker pid=7532) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2


2023-10-22 13:03:35,912	INFO trainable.py:172 -- Trainable.setup took 10.312 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [5]:
N = 3000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()

(RolloutWorker pid=7532) reset triggered
(RolloutWorker pid=7532) reload save for track : 72
(RolloutWorker pid=7532) reset triggered
(RolloutWorker pid=7532) reload save for track : 72


(RolloutWorker pid=7532) c:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.2008818000001611
(RolloutWorker pid=7532)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  0
Saved 0


(RolloutWorker pid=7532) c:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 2.222619200000011
(RolloutWorker pid=7532)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=7532) reset triggered
(RolloutWorker pid=7532) reload save for track : 72


...

(RolloutWorker pid=7532) reset triggered
(RolloutWorker pid=7532) reload save for track : 72


(RolloutWorker pid=7532) c:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16430099990975577
(RolloutWorker pid=7532)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  189


(RolloutWorker pid=7532) c:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 1.8680467999811299
(RolloutWorker pid=7532)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=7532) c:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.20784839990938053
(RolloutWorker pid=7532)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


KeyboardInterrupt: 

In [ ]:
# result = algo.train() #single try